In [ ]:
import os
import time
from abc import ABC, abstractmethod
from pymongo import MongoClient
import requests
import datetime

class AbstractWorker(ABC):

    @abstractmethod
    def fetch_data(self, *args, **kwargs):
        pass

class VKGroupWorker(AbstractWorker):

    def __init__(self, access_token, version="5.131", db_uri="mongodb://localhost:27017", db_name="vk_data"):
        self.access_token = access_token
        self.version = version
        self.base_url = "https://api.vk.com/method/wall.get"
        self.client = MongoClient(db_uri)
        self.db = self.client[db_name]
        self.collection = self.db["posts"]
        self.collection.create_index("id", unique=True)

    def fetch_data(self, group_id, end_time, count=10):
        posts = []
        offset = 0

        params = {
            "owner_id": group_id,
            "access_token": self.access_token,
            "v": self.version,
            "count": count,
            "offset": offset,
            "filter": "owner",
            "end_time": end_time
        }

        response = requests.get(self.base_url, params=params).json()

        if "error" in response:
            print("Error:", response["error"].get("error_msg", "Unknown error"))
            return []

        items = response.get('response', {}).get('items', [])

        for item in items:
            post_data = {
                "id": item.get("id"),
                "date": item.get("date"),
                "text": item.get("text", ""),
                "owner_id": group_id
            }
            self.collection.update_one({"id": post_data["id"]}, {"$set": post_data}, upsert=True)
            posts.append(post_data["text"])

        return posts

    def run_with_time_window(self, group_id, end_time, start_time, stop_time):
        while True:
            current_time = datetime.datetime.now()
            if current_time >= stop_time:
                print(f"[{current_time}] Работа завершена.")
                break

            if current_time >= start_time:
                print(f"[{current_time}] Сбор данных...")
                posts = self.fetch_data(group_id, end_time)

                print(f"[{current_time}] Собрано {len(posts)} новых постов.")
            else:
                print(f"[{current_time}] Ожидание начала работы...")

            time.sleep(120)  #2 минуты

if name == "__main__":
    with open('token.txt') as f:
        access_token = f.read().strip()

    group_id = "lilgarage" 
    end_time = 1734361200  #16.12.2024 15:00:00

    # Настройки работы
    start_time = datetime.datetime(2024, 12, 23, 23, 0, 0)  
    stop_time = datetime.datetime(2024, 12, 23, 23, 10, 0)  

    worker = VKGroupWorker(access_token)
    worker.run_with_time_window(group_id, end_time, start_time, stop_time)